In [1]:
import numpy as np
import itertools
from itertools import permutations
from itertools import product
from linalg_Zpi import linalg_Zpi

class gen_norm1():
    def __init__(self, mod, size): 
        inst = linalg_Zpi(p=mod, n=size)
        field = np.arange(0, mod)
        
        #---------------------------------------------------------------------------        
        # Generate all vectors x such that x*x=1
        def gen_x():
            combine = list(itertools.combinations_with_replacement(field, 2*size))
            store = np.expand_dims(np.zeros(2*size, dtype=int), axis=0)

            for row in combine[1:]:
                sq_sum = 0
                for elem in row:
                    sq_sum += elem**2
                if sq_sum % mod == 1:
                    array = np.array(row)
                    expanded = np.expand_dims(array, axis=0)
                    store = np.append(store, expanded, axis=0)

            permute = np.expand_dims(np.zeros(2*size, dtype=int), axis=0)

            for row in store[1:]:
                p = list(permutations(row, 2*size))
                for item in p:
                    array = np.array(item)
                    expanded = np.expand_dims(array, axis=0)
                    permute = np.append(permute, expanded, axis=0)

            permute = np.unique(permute, axis=0).reshape(-1, size, 2)

            x = np.empty((permute.shape[0], size, 1), dtype=np.csingle)

            for i in range(permute.shape[0]):
                for j in range(size):
                    x[i, j] = complex(permute[i,j,0], permute[i,j,1])

            return x
        
        # Find minimal list of norm-1 vectors
        def sim_x():
            norm0 = gen_x()[1:]
            length = len(norm0)
            
            inds = np.arange(norm0.shape[0])
    
            for i in range(length-1):
                vector = norm0[i]

                for j in range(i+1,length):
                    if inds[j]== -1:
                        continue
                    else:
                        if inst.mult(vector, norm0[j]) != None:
                            inds[j]= -1
            
            return norm0[inds[inds!=-1]]

        self.x = sim_x()
                
        #---------------------------------------------------------------------------     
        # Generate vectors x* from x
        def gen_xstar():
            x_star = np.empty((len(self.x), size), dtype=np.csingle)

            for i in range(len(self.x)):
                x_star[i] = np.transpose(np.conjugate(self.x[i]))
                
            return x_star
        
        self.xstar = gen_xstar()
        
        #--------------------------------------------------------------------------- 
        # Set class variables
        
        self.mod = mod
        self.size = size
        

In [2]:
# Input mod and size values
p = 7
n = 2

In [3]:
# Save norm-0 vectors as a .npz file

string = "norm1/p=" + str(p) + "_n=" + str(n)
np.savez_compressed(string, 
                    norm1 = gen_norm1(mod=p, size=n).x
                   )

In [ ]:
# To retrieve the stored norm-1 vectors from an existing file, we use a command in the following form:
# np.load("norm1/p=7_n=2.npz")["norm1"]